# Simple example demonstrating use of tred library
This is a rough demonstration of the basic functionalities
in `tred`. It unfortunately, at the moment, does not cover all of the public 
functionality. 

In [1]:
import numpy as np
from numpy.testing import assert_allclose

import tred
from tred import display_tensor_facewise as disp

First we will create some dummy data

In [2]:
# let's use a numpy rng
RNG = np.random.default_rng(seed=1234)

# then we will generate some dummy data
n, p, t = 3, 4, 2
X = RNG.random(size=(n, p, t))

Let's quickly view our data using the `display_tensor_facewise` function, which we imported as `disp`. Note: this looks a bit nicer than calling `print(X)`

In [3]:
disp(X)

Tensor with shape (3, 4, 2)
[[[0.9767   0.923246 0.319097 0.241766]
  [0.964079 0.441006 0.863621 0.674881]
  [0.735758 0.172066 0.060139 0.671238]]

 [[0.380196 0.261692 0.118091 0.318534]
  [0.26365  0.609871 0.863758 0.659874]
  [0.222754 0.870415 0.683689 0.611018]]]


## tsvdm tensor decomposition algorithm
Let's get the tsvdm decomposition first, using the `tsvdm` function

In [4]:
# so, first we can use tred's tsvdm decomposition
# NOTE: we return V not Vt
U, S, V = tred.tsvdm(X)

In [5]:
disp(U)

Tensor with shape (3, 3, 2)
[[[ 0.171555 -0.032544 -0.567501]
  [ 0.773763 -0.775055  0.249955]
  [ 1.006132  0.036617  0.276475]]

 [[ 0.496972  1.180533  0.082534]
  [ 0.219636 -0.048142 -0.829184]
  [-0.253649  0.03023   0.919058]]]


In [6]:
disp(S)

Tensor with shape (3, 4, 2)
[[[ 2.490714  0.        0.        0.      ]
  [ 0.        0.951338  0.        0.      ]
  [ 0.        0.        0.335458  0.      ]]

 [[ 1.295923  0.        0.        0.      ]
  [ 0.       -0.060585  0.        0.      ]
  [ 0.        0.        0.109848  0.      ]]]


In [7]:
disp(V)

Tensor with shape (4, 4, 2)
[[[ 0.727706 -0.264466 -0.165429 -0.41692 ]
  [-0.151391 -0.020989 -0.291684  0.607529]
  [-0.017056 -0.610954  0.235092  0.099226]
  [ 0.39763  -0.166782  0.608801  0.521138]]

 [[ 0.024415  0.950528 -0.335921 -0.427048]
  [ 0.843444  0.655176  0.433084  0.440744]
  [ 0.695221 -0.366154 -0.961108  0.142126]
  [ 0.306266 -0.24841   0.487353 -0.882514]]]


In [8]:
# NOTE: the V tensor is not transposed (facewise)!
# in order to do a facewise transpose - use numpy
Vt = V.transpose(1, 0, 2)

In [9]:
disp(Vt)

Tensor with shape (4, 4, 2)
[[[ 0.727706 -0.151391 -0.017056  0.39763 ]
  [-0.264466 -0.020989 -0.610954 -0.166782]
  [-0.165429 -0.291684  0.235092  0.608801]
  [-0.41692   0.607529  0.099226  0.521138]]

 [[ 0.024415  0.843444  0.695221  0.306266]
  [ 0.950528  0.655176 -0.366154 -0.24841 ]
  [-0.335921  0.433084 -0.961108  0.487353]
  [-0.427048  0.440744  0.142126 -0.882514]]]


Let's also verify that we can use the tsvdm decomposition to reconstruct the 
original tensor `X`.

In [10]:
# see documentation - allows us to apply a function over a sequence of inputs 
# cumulatively from left to right
from functools import reduce

# tred has a function to comput the m_product, but it does not make default assumptions
# about the m-transform - as the user, you need to write a simple wrapper to make it work
# with a m-product you want

# you can fetch the default m-transforms used by tsvdm and TPCA using the following tred 
# function
M, Minv = tred.generate_default_m_transform_pair(t)

# then write our wrapper function
def m_product_wrapper(A, B): 
    return tred.m_product(A, B, M=M, Minv=Minv)

# multiply U, S, Vt together using the m-product and default m-transfrom
X_reconstructed = reduce(m_product_wrapper, (U, S, Vt))
assert_allclose(X_reconstructed, X)

Notice how the last column of `S` is all zeroes. This means that some of the
`V` tensor is redundant. We can pass in `full_frontal_slices=False` to only
return us the necessary elements in the tensor.

In [11]:
U2, S2, V2 = tred.tsvdm(X, full_frontal_slices=False)

(U2.shape, S2.shape, V2.shape)

((3, 3, 2), (3, 3, 2), (4, 3, 2))

Sense check that we have not 'loss' any information, and we can still 
fully reconstruct our original tensor

In [12]:
V2t = V2.transpose(1, 0, 2)
X_reconstructed2 = reduce(m_product_wrapper, (U2, S2, V2t))
assert_allclose(X_reconstructed2, X)

## TPCA unsupervised tensor dimensionality reduction algorithm
Now, let's use the tensor decomposition using the `tpca` object. Revise some basics of object oriented programming and scikit-learn's interface if you are coming from R.

In [13]:
# first, create a transformer object
tpca = tred.TPCA()

There's two ways to do this:

1. You explicitly fit your tpca object _first_, and then transform your data.

Let's see what happens if you call `transform` _before_ `fit`.

In [14]:
# ERROR
X_transformed_1 = tpca.transform(X)

NotFittedError: This TPCA instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [15]:
# so, we need to fit the tpca object first - then call transform!
tpca.fit(X)
X_transformed_1 = tpca.transform(X)

Bit of a late update - but `disp` (named `display_tensor_facewise`) actually 
works for matrix and vector inputs too. It just does a bit of rounding and 
prints the size of the input. We have left the unfortunate original name as is, 
to avoid breaking existing workflows. 

In [16]:
# the size of this is n rows by min(n, p) columns
disp(X_transformed_1)

Matrix with shape (3, 6)
[[ 0.56846  0.51181 -0.15435 -0.06899  0.      -0.     ]
 [-0.00667 -0.55972 -0.11783  0.14046 -0.      -0.     ]
 [-0.5618   0.04792  0.27218 -0.07147  0.       0.     ]]


2. Use `fit_transform`

NOTE: this produces the same result to machine precision, but it's not identical, as we 
use a slightly more efficient computation to obtain the transformed data. 

See the `tred` library implementation for more details

In [17]:
X_transformed_2 = tpca.fit_transform(X)

In [18]:
# NOTE: this is the same as the array above - to machine precision
# some numbers might LOOK different, but you'll see that they are tiny and
# are effectively the same
disp(X_transformed_2)

Matrix with shape (3, 6)
[[ 0.56846  0.51181 -0.15435 -0.06899 -0.       0.     ]
 [-0.00667 -0.55972 -0.11783  0.14046 -0.       0.     ]
 [-0.5618   0.04792  0.27218 -0.07147 -0.       0.     ]]


In [19]:
# the tensor operations produce some floating point errors
# below are the tolerances we use in tred's testing module, which show that the two
# transformations of X are indeed the same
RTOL = 1e-7
ATOL = 1e-10
assert_allclose(X_transformed_1, X_transformed_2, rtol=RTOL, atol=ATOL)

We can use the fitted objects to transform out of bag data

In [20]:
# imagine one new observation, with p features, across t time points
new_obs = RNG.random(size=(1, p, t))

In [21]:
disp(new_obs)

Tensor with shape (1, 4, 2)
[[[0.060137 0.438952 0.003132 0.85849 ]]

 [[0.977769 0.532595 0.251267 0.425298]]]


In [23]:
disp(tpca.transform(new_obs))

Matrix with shape (1, 6)
[[-0.13546  0.34662  0.39719 -0.55304  0.95086  0.07885]]


Make sure you do not call `fit` or `fit_transform` on the new data, otherwise 
the `tpca` object will learn new weights!